In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PP

c:\Users\Aryan\anaconda3\envs\mlchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY','bc258987-f513-47af-8090-1e600071e84f')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','gcp-starter')
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [4]:
#Extract data from the PDF 
def load_pdf(data): 
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyMuPDFLoader)

    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("../data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Number of chunks:", len(text_chunks))

Number of chunks: 4021


In [8]:
# text_chunks

In [9]:
#download embedding model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

c:\Users\Aryan\anaconda3\envs\mlchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [14]:
from pinecone import Pinecone

pc = Pinecone(api_key="bc258987-f513-47af-8090-1e600071e84f")
index = pc.Index("ml-chatbot")

In [15]:
print(pc.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'ml-chatbot-xfedrzf.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'ml-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [16]:
pc = PP(api_key=PINECONE_API_KEY)

# Create or connect to an existing index
index_name = 'ml-chatbot'

# docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [17]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query="What is reinforcement learning"

docs=docsearch.similarity_search(query, k=3)

print('Results: ', docs)

Results:  [Document(page_content="as in the case of AlphaZero, which uses tree search to decide on the best action for a given \ngame state. The policy may also be stochastic and assign probabilities to actions, given a \nstate.\nRewards – learning from actions\nThe reward signal is a single value that the environment sends to the agent at each time \nstep. The agent's objective is typically to maximize the total reward received over time. \nRewards can also be a stochastic function of the state and the actions. They are typically"), Document(page_content="increase the reward in the future. RL algorithms optimize the agent's learning using \ndynamical systems theory and, in particular, the\xa0optimal control of Markov decision \nprocesses with incomplete information.\nRL differs from supervised learning, where the training data lays out both the context \nand the correct decision for the algorithm. It is tailored to interactive settings where the \noutcomes only become available over t

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [20]:
from langchain_pinecone import PineconeVectorStore  
text_field = "text"  
vectorstore = PineconeVectorStore(  
    index, embeddings, text_field  
)  

In [22]:
from langchain.chains import RetrievalQA 
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
response = qa(query)
print(response['result'])

 Reinforcement learning (RL) is a subfield of machine learning that focuses on training agents to make decisions in complex, uncertain environments. The goal of RLearning is to learners rewards maximizing an RLearned, an RL is to trainings include maximizing reinforcement learning is to optimize the agent-RL is to trainigrl is to trainin this fielding reinforcement learning in reinforcement learning in RLearningagent reinforement learning is to learn
RL is to maximizing the agent' an RLearning is to train
